In [ ]:
# import libraries
import pandas as pd

In [ ]:
# import data
data_file = './data/WDI/WDIData.csv'
df = pd.read_csv(data_file)

In [ ]:
print(df.shape)
df.head(5)

In [ ]:
# inspect indicators
for i in df['Indicator Name'].unique():
    print(i)

In [ ]:
indicators = ['GDP growth (annual %)', 'GDP per capita (current US$)', 'GDP per capita growth (annual %)', 'GINI index (World Bank estimate)', 'Population density (people per sq. km of land area)', 'Population growth (annual %)', 'Population, total']
indicators

In [ ]:
# filter dataset to keep only indicators of interest
df_subset = df[df['Indicator Name'].isin(indicators)]
df_subset.head(10)

one table per indicator (two indicators) one row per year...field names, country_name, country_code, year, indicator_value

In [ ]:
# Drop unnecessary column
df_subset.drop('Indicator Code', axis=1, inplace=True)
df_subset[df_subset['Country Code'] == 'USA'].head(5)

In [ ]:
# Use melt to transform array of years (1960-2018) to rows (one row per year per indicator)
df_melt = df_subset.melt(id_vars=["Country Name", "Country Code", "Indicator Name"], 
        var_name="Date", 
        value_name="Value")

In [ ]:
df_melt[df_melt['Country Code'] == 'USA'].head(10)

In [ ]:
# Use pivot_table to get a column per indicator
df_pivot = pd.pivot_table(df_melt, values = 'Value', index=['Country Name','Country Code', 'Date'], columns = 'Indicator Name').reset_index()
df_pivot[df_pivot['Country Code'] == 'USA'].head(10)

In [ ]:
# Format values
for col in ['GDP growth (annual %)',
       'GDP per capita (current US$)', 'GDP per capita growth (annual %)',
       'GINI index (World Bank estimate)',
       'Population density (people per sq. km of land area)',
       'Population growth (annual %)']:

    df_pivot[col] = round(df_pivot[col],2)

df_pivot['Population, total'] = df_pivot['Population, total'].astype('int', errors='ignore')

In [ ]:
# Check output
df_pivot[df_pivot['Country Code'] == 'USA'].head()

# Need to fix Population, total column

In [ ]:
# Fix columns name to match DB Scheme
df_pivot.columns = [
    'country', 'country_code', 'year', 'gdp_growth_annual', 
    'gdp_per_cap', 'gdp_per_cap_growth_annual', 'gini_index', 
    'pop_density', 'pop_growth_annual', 'pop_total']

df_pivot.head()